In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

C:\Users\prit4\AppData\Local\Temp\ipykernel_18864\291067091.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dataset_path = "dataset.csv"
df = pd.read_csv(dataset_path)

In [3]:
# Remove negative values from the dataset
for column in df.select_dtypes(include=['int64', 'float64']).columns:
    df = df[df[column] >= 0]

print('Negative values removed from the dataset.')

Negative values removed from the dataset.


In [4]:
# scaler = MinMaxScaler()
# scaled_features = scaler.fit_transform(df[['month', 'day', 'time', 'humidity', 'tempC']])
# print(scaled_features)

In [5]:
scaled_features = pd.DataFrame()
scaled_features['month'] = df['month'] / 12
scaled_features['day'] = df['day'] / 31
scaled_features['time'] = df['time'] / 24
scaled_features['humidity'] = df['humidity'] / 100
scaled_features['tempC'] = df['tempC'] / 50
print(scaled_features.head())

      month       day      time  humidity  tempC
0  0.083333  0.032258  0.000000      0.74   0.36
1  0.083333  0.032258  0.041667      0.78   0.36
2  0.083333  0.032258  0.083333      0.82   0.34
3  0.083333  0.032258  0.125000      0.85   0.34
4  0.083333  0.032258  0.166667      0.83   0.34


In [6]:
X = scaled_features
y = df['fan_speed'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [9]:
model = tf.keras.Sequential([

    tf.keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(alpha=0.2), input_shape=(5,)),
    tf.keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
    tf.keras.layers.Dense(6, activation='softmax')  # Softmax activation for multi-class classification
])

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=90, validation_split=0.2)


Epoch 1/100
63/63 [==============================] - 1s 3ms/step - loss: 1.4663 - accuracy: 0.3957 - val_loss: 1.3017 - val_accuracy: 0.4395
Epoch 2/100
63/63 [==============================] - 0s 1ms/step - loss: 1.2142 - accuracy: 0.4768 - val_loss: 1.1131 - val_accuracy: 0.5619
Epoch 3/100
63/63 [==============================] - 0s 2ms/step - loss: 1.0630 - accuracy: 0.5478 - val_loss: 0.9874 - val_accuracy: 0.6088
Epoch 4/100
63/63 [==============================] - 0s 2ms/step - loss: 0.9568 - accuracy: 0.5938 - val_loss: 0.8967 - val_accuracy: 0.6117
Epoch 5/100
63/63 [==============================] - 0s 1ms/step - loss: 0.8782 - accuracy: 0.6268 - val_loss: 0.8252 - val_accuracy: 0.6366
Epoch 6/100
63/63 [==============================] - 0s 1ms/step - loss: 0.8189 - accuracy: 0.6591 - val_loss: 0.7777 - val_accuracy: 0.6728
Epoch 7/100
63/63 [==============================] - 0s 1ms/step - loss: 0.7686 - accuracy: 0.6816 - val_loss: 0.7438 - val_accuracy: 0.6743
Epoch 8/100
6

In [11]:
# Convert true classes to one-hot encoding
true_classes_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=6)
predicted_classes = model.predict(X_test)
# print(predicted_classes)
# Calculate accuracy
accuracy = np.mean(np.argmax(predicted_classes, axis=1) == np.argmax(true_classes_one_hot, axis=1))
print("Test Accuracy (Threshold):", accuracy)


55/55 [==============================] - 0s 803us/step
Test Accuracy (Threshold): 0.9015367103016505


In [25]:
# model.save('model.h5')

c:\Users\prit4\OneDrive\Desktop\stuff\active_Github_repos\Mini_project\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
